In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

testset_frac = 0.25
data_samples = 100000 

df_full = pd.read_csv('data/Encoded.csv')
df_full = df_full.sample(n=data_samples)
df_full = df_full.iloc[:, 1:]
df_full = df_full.drop(columns=['Attack Tool', 'Label', 'sVid', 'dVid', '54'])
df_full = df_full.fillna(df_full.median())

<ipython-input-1-168249438b94>:45: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_full = df_full.fillna(df_full.median())


In [2]:
learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=3
number_of_slices = 5
isSmote = False
runtime = 21

file_name = "federated_" + str(isSmote) + "_" + str(number_of_slices)  + "_" + str(runtime) + ".txt"
file = open(file_name, "w")

In [3]:
dummies = pd.get_dummies(df_full['Attack Type'])  # Classification
outcomes = dummies.columns
print(outcomes)
num_classes = len(outcomes)
Y = dummies.values
print(df_full['Attack Type'].value_counts())
print(num_classes)

Index(['Benign', 'HTTPFlood', 'ICMPFlood', 'SYNFlood', 'SYNScan',
       'SlowrateDoS', 'TCPConnectScan', 'UDPFlood', 'UDPScan'],
      dtype='object')
Benign            38705
UDPFlood          37931
HTTPFlood         11733
SlowrateDoS        6126
SYNScan            1671
TCPConnectScan     1612
UDPScan            1333
SYNFlood            800
ICMPFlood            89
Name: Attack Type, dtype: int64
9


In [4]:
df_Benign = df_full[df_full['Attack Type']=='Benign']
df_HTTPFlood= df_full[df_full['Attack Type']=='HTTPFlood']
df_UDPFlood= df_full[df_full['Attack Type']=='UDPFlood']
df_SYNFlood= df_full[df_full['Attack Type']=='SYNScan']
df_SlowrateDoS= df_full[df_full['Attack Type']=='SlowrateDoS']

df_Benign.loc[(df_Benign['Attack Type'] == 'Benign'), 'Attack Type'] = 0
df_HTTPFlood.loc[(df_HTTPFlood['Attack Type'] == 'HTTPFlood'), 'Attack Type'] = 2
df_UDPFlood.loc[(df_UDPFlood['Attack Type'] == 'UDPFlood'), 'Attack Type'] = 1
df_SYNFlood.loc[(df_SYNFlood['Attack Type'] == 'SYNScan'), 'Attack Type'] = 4
df_SlowrateDoS.loc[(df_SlowrateDoS['Attack Type'] == 'SlowrateDoS'), 'Attack Type'] = 3

print(df_Benign.shape)
print(df_HTTPFlood.shape)
print(df_UDPFlood.shape)
print(df_SYNFlood.shape)
print(df_SlowrateDoS.shape)

df_filterd = pd.concat([df_Benign,df_HTTPFlood,df_UDPFlood,df_SYNFlood,df_SlowrateDoS])
print(df_filterd['Attack Type'].value_counts())
print(df_filterd.shape)
type_df = df_filterd['Attack Type'].copy()
data_df = df_filterd.drop('Attack Type',axis=1)
print(type_df.shape)


data_df = data_df / data_df.max()
df_normalized = pd.concat([data_df,type_df],axis=1)

(38705, 90)
(11733, 90)
(37931, 90)
(1671, 90)
(6126, 90)
0    38705
1    37931
2    11733
3     6126
4     1671
Name: Attack Type, dtype: int64
(96166, 90)
(96166,)


In [5]:
def divide_train_test(df, propotion=0.1):
    
    df_train = []
    df_test = []
    for key,val in df['Attack Type'].value_counts().iteritems():
        df_part = df[df['Attack Type'] == key]
        df_part = df_part.dropna(axis=1)
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train.append(df_part[int(df_part.shape[0]*propotion):df_part.shape[0]])
        
    return df_train,df_test

In [6]:
def get_data_for_slices(df_train, number_of_slices, isSmote=False, x_name="x_train", y_name="y_train"):
    
    x_data_dict= dict()
    y_data_dict= dict()    
    
    for i in range(number_of_slices):
        xname= x_name+str(i)
        yname= y_name+str(i)
        df_types = []
        
        for df in df_train:
            df_type = df[int(df.shape[0]*i/number_of_slices):int(df.shape[0]*(i+1)/number_of_slices)]
            df_types.append(df_type)
        
        slice_df = pd.concat(df_types)
        y_info = slice_df.pop('Attack Type').values
        x_info = slice_df.values
        y_info = y_info.astype('int')
        
        if isSmote:
            sm = SMOTE(random_state=42)
            x_info, y_info = sm.fit_resample(x_info, y_info)
        
        print('========================================================================================')
        print('\tX part size for slice ' + str(i) + ' is ' + str(x_info.shape))
        print('\tY part size for slice ' + str(i) + ' is ' + str(y_info.shape))
        print('Value types of each class in slice : ' + str(i))
        print(np.unique(y_info,return_counts=True))
        
        x_info = torch.tensor(x_info).float()
        y_info = torch.tensor(y_info).type(torch.LongTensor)
            
        x_data_dict.update({xname : x_info})
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict     

In [7]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)
print(len(df_train))
# print('Value counts in train set : ')
# df_train['Attack Type'].value_counts()
# print('Value counts in test set : ')
# print(df_test['Attack Type'].value_counts())

x_train_dict, y_train_dict = get_data_for_slices(df_train, number_of_slices, isSmote)

df_test = pd.concat(df_test)
y_test = df_test.pop('Attack Type').values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

inputs = x_test.shape[1]
outputs = 5
print(inputs,outputs)

5
	X part size for slice 0 is (17308, 86)
	Y part size for slice 0 is (17308,)
Value types of each class in slice : 0
(array([0, 1, 2, 3, 4]), array([6967, 6827, 2112, 1102,  300], dtype=int64))
	X part size for slice 1 is (17311, 86)
	Y part size for slice 1 is (17311,)
Value types of each class in slice : 1
(array([0, 1, 2, 3, 4]), array([6967, 6828, 2112, 1103,  301], dtype=int64))
	X part size for slice 2 is (17310, 86)
	Y part size for slice 2 is (17310,)
Value types of each class in slice : 2
(array([0, 1, 2, 3, 4]), array([6967, 6827, 2112, 1103,  301], dtype=int64))
	X part size for slice 3 is (17311, 86)
	Y part size for slice 3 is (17311,)
Value types of each class in slice : 3
(array([0, 1, 2, 3, 4]), array([6967, 6828, 2112, 1103,  301], dtype=int64))
	X part size for slice 4 is (17311, 86)
	Y part size for slice 4 is (17311,)
Value types of each class in slice : 4
(array([0, 1, 2, 3, 4]), array([6967, 6828, 2112, 1103,  301], dtype=int64))
Test set size is : x => (9615, 86

In [8]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [9]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [10]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [11]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [12]:
def confusion_mat(model, test_loader):
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
    precisionv = precision_score(y_true,y_pred,average='macro')
    recallv = recall_score(y_true,y_pred,average='macro')
    print('precision value: '+str(precisionv))
    print('recall value: '+ str(recallv))
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

In [13]:
def create_model_optimizer_criterion_dict(number_of_slices):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_slices):
        model_name="model"+str(i)
        model_info=Net2nn(inputs, outputs)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [14]:
def get_averaged_weights(model_dict, number_of_slices):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_slices):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_slices
        fc1_mean_bias = fc1_mean_bias/ number_of_slices
    
        fc2_mean_weight =fc2_mean_weight/number_of_slices
        fc2_mean_bias = fc2_mean_bias/ number_of_slices
    
        fc3_mean_weight =fc3_mean_weight/number_of_slices
        fc3_mean_bias = fc3_mean_bias/ number_of_slices
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [15]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_slices=number_of_slices)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [16]:
def compare_local_and_merged_model_performance(number_of_slices):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_slices,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_slices):
    
        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [17]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices):
    with torch.no_grad():
        for i in range(number_of_slices):
            print('Updating model :' + name_of_models[i] )
            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

In [18]:
def start_train_end_node_process(number_of_slices):
    for i in range (number_of_slices): 

        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [19]:

def start_train_end_node_process_without_print(number_of_slices):
    for i in range (number_of_slices): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)

In [20]:
def start_train_end_node_process_print_some(number_of_slices, print_amount):
    for i in range (number_of_slices): 
        
        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], 
                                 y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))    

In [21]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()

In [22]:
print("------ Centralized Model ------")

train_acc = []
test_acc = []
train_loss = []
test_loss = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

for i in range(number_of_slices):
    centralized_model = Net2nn(inputs,outputs)
    centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
    centralized_criterion = nn.CrossEntropyLoss()
#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
    print('Training with slice ' + str(i+1) + ' data' )
    x_name = 'x_train' + str(i)
    y_name = 'y_train' + str(i)
    train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    for epoch in range(numEpoch):
        central_train_loss, central_train_accuracy = train(centralized_model, train_dl, centralized_criterion, centralized_optimizer)
        central_test_loss, central_test_accuracy = validation(centralized_model, test_dl, centralized_criterion)
        
        train_acc.append(central_train_accuracy)
        train_loss.append(central_train_loss)
        test_acc.append(central_test_accuracy)
        test_loss.append(central_test_loss)
        
        print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    confusion_mat(centralized_model, test_dl)
    
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
print('Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))


------ Centralized Model ------
Training with slice 1 data
epoch:   1 | train accuracy:  0.8293 | test accuracy:  0.9716
epoch:   2 | train accuracy:  0.9720 | test accuracy:  0.9790
epoch:   3 | train accuracy:  0.9791 | test accuracy:  0.9806
epoch:   4 | train accuracy:  0.9812 | test accuracy:  0.9811
epoch:   5 | train accuracy:  0.9811 | test accuracy:  0.9797
epoch:   6 | train accuracy:  0.9839 | test accuracy:  0.9882
epoch:   7 | train accuracy:  0.9841 | test accuracy:  0.9886
epoch:   8 | train accuracy:  0.9852 | test accuracy:  0.9793
epoch:   9 | train accuracy:  0.9860 | test accuracy:  0.9878
epoch:  10 | train accuracy:  0.9872 | test accuracy:  0.9877
epoch:  11 | train accuracy:  0.9863 | test accuracy:  0.9809
epoch:  12 | train accuracy:  0.9872 | test accuracy:  0.9903
epoch:  13 | train accuracy:  0.9871 | test accuracy:  0.9901
epoch:  14 | train accuracy:  0.9854 | test accuracy:  0.9878
epoch:  15 | train accuracy:  0.9880 | test accuracy:  0.9794
epoch:  16 

In [23]:
file.write('\nCentralized Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
file.write('\nCentralized Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))

51

In [24]:
print(x_train_dict["x_train1"].shape, y_train_dict["y_train1"].shape)
print(x_test.shape, y_test.shape)

torch.Size([17311, 86]) torch.Size([17311])
torch.Size([9615, 86]) torch.Size([9615])


In [25]:
main_model = Net2nn(inputs,outputs)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

In [26]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_slices)

In [27]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

print(name_of_x_train_sets)
print(name_of_y_train_sets)
print("\n ------------")
print(name_of_models)
print(name_of_optimizers)
print(name_of_criterions)

['x_train0', 'x_train1', 'x_train2', 'x_train3', 'x_train4']
['y_train0', 'y_train1', 'y_train2', 'y_train3', 'y_train4']

 ------------
['model0', 'model1', 'model2', 'model3', 'model4']
['optimizer0', 'optimizer1', 'optimizer2', 'optimizer3', 'optimizer4']
['criterion0', 'criterion1', 'criterion2', 'criterion3', 'criterion4']


In [28]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0318, -0.0075, -0.0096, -0.0560,  0.0213]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0103,  0.0375, -0.0418, -0.0335, -0.0655]],
       grad_fn=<SliceBackward>)


In [29]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4


In [30]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0318, -0.0075, -0.0096, -0.0560,  0.0213]],
       grad_fn=<SliceBackward>)
tensor([[-0.0318, -0.0075, -0.0096, -0.0560,  0.0213]],
       grad_fn=<SliceBackward>)


In [31]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_slices, print_amount)

Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.82037 | test accuracy: 0.97098
epoch:   2 | train accuracy: 0.97302 | test accuracy: 0.97608
epoch:   3 | train accuracy: 0.97706 | test accuracy: 0.98294
epoch:   4 | train accuracy: 0.98128 | test accuracy: 0.97733
epoch:   5 | train accuracy: 0.98313 | test accuracy: 0.98461
epoch:   6 | train accuracy: 0.98134 | test accuracy: 0.98918
epoch:   7 | train accuracy: 0.98492 | test accuracy: 0.98305
epoch:   8 | train accuracy: 0.98480 | test accuracy: 0.98721
epoch:   9 | train accuracy: 0.98683 | test accuracy: 0.99012
epoch:  10 | train accuracy: 0.98804 | test accuracy: 0.98513
epoch:  11 | train accuracy: 0.98700 | test accuracy: 0.98690
epoch:  12 | train accuracy: 0.98717 | test accuracy: 0.99064
epoch:  13 | train accuracy: 0.98792 | test accuracy: 0.98544
epoch:  14 | train accuracy: 0.98735 | test accuracy: 0.98762
epoch:  15 | train accuracy: 0.98810 | test accuracy: 0.99106
epoch:  16 | train accuracy: 0

In [32]:
## As you can see, wieghts of local models are updated after training process
print(main_model.fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])

tensor([-0.0318, -0.0075, -0.0096, -0.0560,  0.0213], grad_fn=<SliceBackward>)
tensor([-0.0458,  0.1004,  0.0328,  0.0410, -0.0149], grad_fn=<SliceBackward>)


In [33]:
before_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nbefore training main model')
confusion_mat(main_model, test_dl)

main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 

after_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nafter training main model')
confusion_mat(main_model, test_dl)

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.003474100270438943
recall value: 0.2
confusion matrix for normal scenario for slices : 5
[[   0    1    0    0 3869]
 [   0    0    0    0 3793]
 [   0    0    0    0 1173]
 [   0    0    0    0  612]
 [   0    0    0    0  167]]
precision value: 0.9809979335673242
recall value: 0.9855273779439999
confusion matrix for normal scenario for slices : 5
[[3837   14    6    9    4]
 [   5 3788    0    0    0]
 [   0    0 1161   12    0]
 [   0    0   32  580    0]
 [   0    0    0    0  167]]


In [34]:
print("Federated main model vs individual local models before FedAvg first iteration")
file.write('\nBefore training federated')
file.write('\n'+str(before_acc_table))
before_acc_table

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.9880,0.0174
1,sample 1,0.9893,0.0174
2,sample 2,0.9900,0.0174
3,sample 3,0.9856,0.0174
4,sample 4,0.9907,0.0174


In [35]:
print("Federated main model vs individual local models after FedAvg first iteration")
file.write('\nAfter training federated')
file.write('\n'+str(after_acc_table))
after_acc_table

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.9880,0.9915
1,sample 1,0.9893,0.9915
2,sample 2,0.9900,0.9915
3,sample 3,0.9856,0.9915
4,sample 4,0.9907,0.9915


In [36]:
print("Before 1st iteration main model accuracy on all test data: {:7.4f}".format(before_test_accuracy))
print("After 1st iteration main model accuracy on all test data: {:7.4f}".format(after_test_accuracy))
print("Centralized model accuracy on all test data: {:7.4f}".format(central_test_accuracy))

Before 1st iteration main model accuracy on all test data:  0.0174
After 1st iteration main model accuracy on all test data:  0.9915
Centralized model accuracy on all test data:  0.9884


In [37]:
for i in range(2):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_without_print(number_of_slices)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy)) 
    confusion_mat(main_model, test_dl)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 2 : main_model accuracy on all test data:  0.9930
precision value: 0.981775585202206
recall value: 0.9899070163786196
confusion matrix for normal scenario for slices : 5
[[3847    6    4    9    4]
 [   8 3785    0    0    0]
 [   0    0 1152   21    0]
 [   0    0   15  597    0]
 [   0    0    0    0  167]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 3 : main_model accuracy on all test data:  0.9947
precision value: 0.9867520515447165
recall value: 0.9917646645718019
confusion matrix for normal scenario for slices : 5
[[3852    5    5    6    2]
 [   3 3790    0    0    0]
 [   0    0 1156   17    0]
 [   0    0   13  599    0]
 [   0    0    0    0  167]]


In [38]:
confusion_mat(main_model, test_dl)

precision value: 0.9867520515447165
recall value: 0.9917646645718019
confusion matrix for normal scenario for slices : 5
[[3852    5    5    6    2]
 [   3 3790    0    0    0]
 [   0    0 1156   17    0]
 [   0    0   13  599    0]
 [   0    0    0    0  167]]
